<a href="https://colab.research.google.com/github/garciafranciscomartn/nextflow_intro_rsg/blob/main/Modulo4_Containers_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Containers*
Ya terminamos de repasar los componentes básicos de Nextflow para crear un flujo de trabajo simple capaz de procesar y evaluar la calidad de archivos fastq, paralelizar la ejecución si hay múltiples archivos y recopilar los resultados para su posterior análisis.


Sin embargo, hubo un detalle que no explicamos de los procesos, y fue que estábamos usando containers para emplear las diferentes herramientas. Normalmente, necesitaría instalar estas herramientas, administrar sus dependencias y resolver cualquier conflicto.


Todo esto es muy tedioso y molesto, por vamos a ver cómo usar *containers* para resolver este problema de forma mucho más práctica.


Un *containers* es una unidad de software ligera, independiente y ejecutable, creada a partir de una imagen de *containers* que incluye todo lo necesario para ejecutar una aplicación, incluyendo código, bibliotecas del sistema y configuración.


En nuestro trabajo vamos a trabajar con *Conda* y *Docker*, pero Nextflow ofrece varias opciones más que pueden revisar con este [link](https://www.nextflow.io/docs/latest/container.html).


[Docker](https://www.docker.com/) es una plataforma que permite crear, distribuir y ejecutar containers de forma sencilla. Un container Docker es como una "caja" que contiene el programa que queremos usar (por ejemplo, fastqc o multiqc), junto con todas sus dependencias, sistema operativo mínimo y configuraciones necesarias. Esto nos asegura que nuestro pipeline se va a comportar igual en cualquier computadora, sin importar el sistema operativo o los paquetes instalados.


[Conda](https://anaconda.org/anaconda/conda), por otro lado, es un sistema de gestión de paquetes y entornos que permite instalar programas y todas sus dependencias dentro de un entorno aislado. A diferencia de Docker, Conda no encapsula todo el sistema operativo, pero es más liviano y muy útil para bioinformática, donde muchos paquetes están disponibles en canales como Bioconda.


Nextflow permite usar ambos enfoques con solo unas pocas líneas de configuración.
- Para Docker, basta con definir el nombre de la imagen que queremos usar, y Nextflow se encarga de ejecutarlo en el container. La sintaxis es ```container <NOMBRE_DE_LA_IMAGEN>```
-  Para Conda, podemos definir un environment.yml o simplemente una lista de paquetes necesarios, y Nextflow los instalará automáticamente (si tenemos conda instalado). La sintaxis es ```conda <NOMBRE_DEL_PAQUETE>```


La página de [Seqera](seqera.io) tiene la sección [Seqera Containers](seqera.io/containers) que permite obtener todas las imágenes de las herramientas que desee usar en una sola imagen. Para eso hay que hacer lo siguiente:
1.  Accedé a la página de Seqera Containers
2.  En la barra de búsqueda, ingresá los paquetes que necesitás, usando el mismo formato que en tu environment.yml (por ejemplo bioconda::fastqc=0.11.9, bioconda::fastp=1.0.1 o bioconda::multiqc=1.9)
3.  Una vez agregados los paquetes, seleccioná "Get Container". Seqera generará automáticamente una imagen Docker con esas dependencias, para la arquitectura seleccionada (linux/amd64 o arm64)
4.  Copiá la URI / nombre de la imagen generada, que tendrá un formato tipo:


```
community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8
```
¿Puedo usar Conda y Docker juntos?


Sí, Nextflow permite definir tanto conda como container en el mismo proceso. Esto resulta útil si querés que un workflow sea flexible y funcione tanto con Docker como con Conda, dependiendo del entorno de ejecución. Por ejemplo:
```
process runFastQC {
   conda "bioconda::fastqc=0.11.9"
   container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8"


   publishDir "${params.outdir}", mode: 'copy'


   input:
   path fastq_file
  
   output:
   path "*.html", emit: fastqc_report
   path "*.zip", emit: fastqc_zip


   script:
   """
   fastqc ${fastq_file}
   """
}  
```
Luego puedo usar perfiles (-profile) para seleccionar Docker o Conda ya que Nextflow permite definir profiles en el archivo nextflow.config. Esto te permite elegir, en tiempo de ejecución, si querés usar Docker, Conda, Singularity, etc., sin tener que modificar el script.


Por ejemplo, en nextflow.config podrías tener:
```
profiles {
 docker {
   docker.enabled = true
 }


 conda {
   conda.enabled = true
 }
}
```
Entonces, al ejecutar tu workflow, podés elegir el perfil:


```
nextflow run hola_fastqc.nf -profile docker
```


o bien:
```
nextflow run hola_fastqc.nf -profile conda
```


Esto le indica a Nextflow que use el entorno adecuado para ejecutar los procesos, respetando las declaraciones de container o conda dentro de cada uno.
Si un proceso tiene ambos (conda y container), Nextflow prioriza el método que corresponde al perfil que seleccionaste. Si usás -profile docker, va a usar Docker; si usás -profile conda, va a usar Conda.


Y ahora que ya introducimos el archivo nextflow.config vamos a profundizar en este.


---
### Manejo de configuraciones y parámetros - ***nextflow.config***
Hasta ahora, estuvimos escribiendo nuestro workflow directamente en archivos .nf, definiendo procesos, canales y lógica de ejecución. Sin embargo, a medida que los pipelines crecen, es mejor separar la configuración del código para mantener todo más ordenado, reutilizable y fácil de adaptar a diferentes entornos (de la misma forma que hicimos con los módulos).


Para eso sirve el archivo nextflow.config


Este archivo te permite:


-  Definir parámetros (como rutas de entrada o nombres de muestras),
-  Establecer cuántos recursos usar por proceso (CPU, memoria, tiempo),
-  Activar Conda, Docker o cualquier otro sistema que elija
-  Definir perfiles (-profile) para adaptar tu workflow a diferentes sistemas o entornos (local, HPC, cloud, etc.)


**IMPORTANTE** el archivo nextflow.config que se coloca en el mismo directorio que el script .nf. y se usa el nombre ***nextflow.config*** para que automáticamente nextflow lo use.


#### Definir parámetros globales
En lugar de escribir valores "duros" en tu script, podés definir parámetros reutilizables en el archivo de configuración. Por ejemplo en nuestro nextflow.config usamos los parámetros definidos en el script original, en la sección **params**:
```
params {
 fastq = "fastq.csv"
 outdir = "results"
}
```
En el script hola-fastq.nf, los usamos como los teniamos, params.fastq y params.outdir, también podés sobreescribir los parámetros al ejecutar el workflow como habíamos visto antes


#### Asignar recursos a procesos


Nextflow permite controlar cuántos recursos usa cada proceso, directamente desde el nextflow.config.


Por ejemplo ejemplo:
```
process {
 withName: runFastQC {
   cpus = 2
   memory = '2 GB'
   time = '1h'
 }
}
```
Esto asegura que el proceso runFastQC usará 2 núcleos, 2 GB de RAM y hasta 1 hora de tiempo de ejecución (ideal para clusters o entornos con reglas de scheduling).


También podés usar withLabel para agrupar procesos que comparten requerimientos:
```
process {
 withLabel: 'qc' {
   cpus = 2
   memory = '3 GB'
 }
}
```
Y en tu script .nf, asignar el label:
```


process runFastp {
 label 'qc'
 ...
}


```
#### Perfiles (-profile) para diferentes entornos


Los profiles permiten adaptar la configuración a distintos entornos (tu laptop, un servidor con SLURM, la nube, etc.), sin modificar el script.
Un ejemplo básico de perfiles en nextflow.config sería:
```
profiles {
 local {
   process.executor = 'local'
 }


 slurm {
   process.executor = 'slurm'
   process.queue = 'bioinfo'
 }


 docker {
   docker.enabled = true
 }


 conda {
   conda.enabled = true
 }
}
```
Después podés ejecutar el workflow con varios perfiles como vimos antes, y hasta combinarlos, por ejemplo podría correr en Slurm usando container de Docker agregando los separados por comma ```-profile slurm,docker```











